# ResNet implementation

[Original video](https://youtu.be/DkNIBBBvcPs)

[GoogLeNet paper](https://arxiv.org/abs/1512.03385)

In [33]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

In [34]:
# ResNet architecture
class block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn(x)
        x = self.relu(x)

        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)

        return x


class ResNet(nn.Module):  # For ResNet50 layers = [3, 4, 6, 3]
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNet layers
        self.layer1 = self._make_layer(block, layers[0], out_channels=64, stride=1)
        self.layer2 = self._make_layer(block, layers[1], out_channels=128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], out_channels=256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], out_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*4, num_classes)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

    def _make_layer(self, block, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []

        # Make input and output tensors identical by size (stride) and channels
        if stride != 1 or self.in_channels != out_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels*4, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels*4))
        
        layers.append(block(self.in_channels, out_channels, identity_downsample, stride))
        self.in_channels = out_channels * 4

        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)


def ResNet50(in_channels=3, num_classes=1000):
    return ResNet(block, [3, 4, 6, 3], in_channels, num_classes)

def ResNet101(in_channels=3, num_classes=1000):
    return ResNet(block, [3, 4, 23, 3], in_channels, num_classes)

def ResNet152(in_channels=3, num_classes=1000):
    return ResNet(block, [3, 8, 36, 3], in_channels, num_classes)

In [35]:
x = torch.randn(64, 1, 32, 32)

model = ResNet50(in_channels=1, num_classes=10)
print(model(x).shape)

model = ResNet101(in_channels=1, num_classes=10)
print(model(x).shape)

model = ResNet152(in_channels=1, num_classes=10)
print(model(x).shape)

torch.Size([64, 10])
torch.Size([64, 10])
torch.Size([64, 10])


## Train the model

In [36]:
# Hyperparameters
in_channels = 1
num_classes = 10
learning_rate = 1e-4
batch_size = 1024
num_epochs = 5  # 40

load_model = True
filename = 'my_checkpoint.pth.tar'

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set transforms
my_transforms = transforms.Compose([
    transforms.Pad(2),
    transforms.ToTensor(),
])

# For the error: HTTPError: HTTP Error 403: Forbidden
# StackOverflow: https://stackoverflow.com/a/66461122/7550928
from six.moves import urllib    
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

# Load data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=my_transforms, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=my_transforms, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize network
model = ResNet50(in_channels=in_channels, num_classes=num_classes).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)


def save_checkpoint(state, filename=filename):
    print(' => Saving checkpoint')
    torch.save(state, filename)


def load_checkpoint(checkpoint):
    print(' => Loading checkpoint')
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    best_acc = checkpoint['acc']


if load_model and os.path.exists(filename):
    load_checkpoint(torch.load(filename))
else:
    best_acc = 0


# Check accuracy on training and test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data', end='')
    else:
        print('Checking accuracy on test data', end='')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)  # shape = 64x10
            _, predictions = scores.max(dim=1)  # get index of max value
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        acc = float(num_correct) / float(num_samples) * 100
        #print(f'Got {num_correct} / {num_samples} with accuracy {acc:.2f}')

        model.train()
        return acc

 => Loading checkpoint


In [37]:
# Train network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to Cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)  # shape 64x10
        loss = criterion(scores, targets)
        losses.append(loss.item())

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()
    
    mean_loss = sum(losses)/len(losses)
    acc = check_accuracy(test_loader, model)
    msg = f'\rLoss at epoch {epoch} is {mean_loss:.5f}. Accuracy is {acc:.2f}'

    if best_acc < acc:
        best_acc = acc
        checkpoint = {'state_dict': model.state_dict(),
                      'optimizer': optimizer.state_dict(),
                      'acc': best_acc}
        print(msg, end='')
        save_checkpoint(checkpoint)
    else:
        print(msg)

Loss at epoch 0 is 0.00580. Accuracy is 96.27
Loss at epoch 1 is 0.00077. Accuracy is 93.86
Loss at epoch 2 is 0.00029. Accuracy is 94.68
Loss at epoch 3 is 0.00021. Accuracy is 94.34
Loss at epoch 4 is 0.00015. Accuracy is 94.26


In [38]:
# MNIST accuracy for:
#   LeNet            = 97.38 %
#   VGG16 pretrained = 97.75 %
#   VGG16            = 98.73 %
#   GoogLeNet        = 99.51 %
#   ResNet101        = 96.64 % (very slow)
print(f': {check_accuracy(train_loader, model):.2f}')
print(f': {check_accuracy(test_loader, model):.2f}')

Checking accuracy on training data: 94.47
Checking accuracy on test data: 94.26
